<a href="https://colab.research.google.com/github/Prathamesh-kadam/GNN-Model/blob/main/for_112_new_data_(11_02_2026).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CELL 1: ALL CfA3+SH0ES PARAMETERS (25+ FEATURES!)
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("🔄 LOADING CfA3+SH0ES - ALL PARAMETERS VERSION")

# Load data
df = pd.read_excel("/content/cfa3_ubvri_cleaned.xlsx")
print(f"📊 Raw dataset: {df.shape}")

# ✅ ALL NUMERIC FEATURES (25+ parameters!)
target_col = 'B_abs_mag'
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [col for col in numeric_cols if col != target_col]

print(f"🔍 ALL numeric columns ({len(numeric_cols)}): {numeric_cols[:10]}...")
print(f"✅ ALL FEATURES ({len(feature_cols)}): {feature_cols[:12]}...")

# KNN IMPUTATION for ALL features
print("\n🧹 KNN IMPUTATION (ALL parameters)...")
imputer = KNNImputer(n_neighbors=5, weights='distance')
df_imputed = pd.DataFrame(
    imputer.fit_transform(df[numeric_cols]),
    columns=numeric_cols,
    index=df.index
)

# Clean dataset with ALL features
df_final = df_imputed[feature_cols + [target_col]].copy()
print(f"✅ Final clean data: {df_final.shape}")

# TRAIN/TEST SPLIT
train_df, test_df = train_test_split(df_final, test_size=0.3, random_state=42)
print(f"📊 SPLIT: Train={len(train_df)} | Test={len(test_df)}")

# TENSORS WITH ALL FEATURES
train_features = torch.tensor(train_df[feature_cols].values, dtype=torch.float32)
train_target  = torch.tensor(train_df[target_col].values, dtype=torch.float32)
test_features  = torch.tensor(test_df[feature_cols].values, dtype=torch.float32)
test_target   = torch.tensor(test_df[target_col].values, dtype=torch.float32)

print(f"\n✅ FINAL DIMENSIONS:")
print(f"   feature_cols: {len(feature_cols)} parameters")
print(f"   train_features: {train_features.shape}")  # [N_train, 25+]
print(f"   train_target:   {train_target.shape}")
print(f"   test_features:  {test_features.shape}")
print(f"   test_target:    {test_target.shape}")

# PHYSICS CHECK
print(f"\n🔬 PHYSICS VALIDATION (key correlations):")
key_params = ['B_dm15', 'MU_SH0ES', 'MWEBV']
for param in key_params:
    if param in feature_cols:
        corr = train_df[param].corr(train_df[target_col])
        print(f"   {param}-B_abs_mag: {corr:.3f}")

print(f"\n🎯 READY: Predict B_abs_mag from {len(feature_cols)} ALL CfA3+SH0ES features!")
print(f"📈 Features include: UBVRI magnitudes, errors, distances, decline rates, extinction...")
print(f"🚀 MAXIMUM PHYSICS for GNN!")


🔄 LOADING CfA3+SH0ES - ALL PARAMETERS VERSION
📊 Raw dataset: (112, 26)
🔍 ALL numeric columns (25): ['MU_SH0ES', 'MU_ERR', 'MWEBV', 'B_app_peak', 'B_app_peak_err', 'B_abs_mag', 'B_abs_mag_err', 'V_app_peak', 'V_app_peak_err', 'V_abs_mag']...
✅ ALL FEATURES (24): ['MU_SH0ES', 'MU_ERR', 'MWEBV', 'B_app_peak', 'B_app_peak_err', 'B_abs_mag_err', 'V_app_peak', 'V_app_peak_err', 'V_abs_mag', 'V_abs_mag_err', 'B_dm15', 'B_dm15_err']...

🧹 KNN IMPUTATION (ALL parameters)...
✅ Final clean data: (112, 25)
📊 SPLIT: Train=78 | Test=34

✅ FINAL DIMENSIONS:
   feature_cols: 24 parameters
   train_features: torch.Size([78, 24])
   train_target:   torch.Size([78])
   test_features:  torch.Size([34, 24])
   test_target:    torch.Size([34])

🔬 PHYSICS VALIDATION (key correlations):
   B_dm15-B_abs_mag: 0.023
   MU_SH0ES-B_abs_mag: -0.480
   MWEBV-B_abs_mag: 0.046

🎯 READY: Predict B_abs_mag from 24 ALL CfA3+SH0ES features!
📈 Features include: UBVRI magnitudes, errors, distances, decline rates, extinction

In [2]:
# CELL 2: FIXED - MAXIMUM SN Ia PHYSICS TEMPLATES (10,000+ FORMULAS)
from itertools import combinations, product, permutations
import random

print("🔬 Generating MAXIMUM SN Ia physics formulas...")

def generate_maximum_physics_templates(params, n_templates=10000):
    """Generate 10,000+ SN Ia empirical formulas - COMPLETE COVERAGE"""
    templates = set()  # Use set to avoid duplicates

    print(f"📊 MAX parameters: {params}")
    print("🔬 Generating ALL combinations...")

    # 1. SINGLE PARAMETER (ALL transformations)
    print("   1. Single parameters...")
    for param in params:
        templates.update([
            f"a*{param}",
            f"a*({param}-1.0)",
            f"a*({param}-1.1)",
            f"a*log({param}+1)",
            f"a*log10({param}+1)",
            f"a*exp({param}/5)",
            f"a*{param}**0.5",
            f"a*{param}**2",
            f"a*abs({param}-18)",
            f"a*{param}**0.33"
        ])

    # 2. TWO-PARAMETER (ALL 5×4=20 pairs × 12 operations)
    print("   2. Two-parameter physics...")
    for p1, p2 in permutations(params, 2):  # All ordered pairs
        templates.update([
            f"a*{p1}+b*{p2}",
            f"a*{p1}-b*{p2}",
            f"a*{p1}*{p2}",
            f"a*{p1}/{p2}",
            f"a*log({p1})+b*{p2}",
            f"a*{p1}*{p2}**0.5",
            f"a*({p1}/{p2})",
            f"a*{p1}**2*{p2}",
            f"a*{p1}*log({p2})",      # ✅ FIXED: log({p2})
            f"a*({p1}-{p2})**0.5",
            f"a*{p1}*exp({p2}/10)",
            f"a*min({p1},{p2})"
        ])

    # 3. THREE-PARAMETER (ALL triplets × 8 forms)
    print("   3. Three-parameter standardization...")
    for combo in combinations(params, min(3, len(params))):
        for perm in permutations(combo):
            p1, p2, p3 = perm
            templates.update([
                f"a*{p1}+b*{p2}+c*{p3}",
                f"a*{p1}*{p2}+b*{p3}",
                f"a*{p1}+b*log({p2})+c*{p3}",
                f"a*({p1}/{p2})+b*{p3}",
                f"a*{p1}*{p2}*{p3}**0.5",
                f"a*{p1}+b*({p2}-{p3})",
                f"a*log({p1}/{p2})+b*{p3}",
                f"a*{p1}**0.5*{p2}+c*{p3}"
            ])

    # 4. FOUR+ PARAMETER (key physics combinations)
    print("   4. Multi-parameter...")
    if len(params) >= 4:
        for combo in combinations(params, 4):
            p1, p2, p3, p4 = combo
            templates.update([
                f"a*{p1}+b*{p2}+c*{p3}+d*{p4}",
                f"a*{p1}*{p2}+b*{p3}+c*{p4}"
            ])

    # 5. SN Ia SPECIFIC (your exact parameters)
    print("   5. SN Ia physics...")
    snia_templates = [
        f"a*B_dm15+b*MU_SH0ES+c*MWEBV",
        f"a*B_dm15+b*(MU_SH0ES-MWEBV)",
        f"a*B_dm15*log(MU_SH0ES)+b*V_abs_mag",
        f"a*(B_app_peak-MU_SH0ES)+b*B_dm15",
        f"a*B_dm15+b*(U_abs_mag-V_abs_mag)",
        f"a*V_abs_mag+b*(B_dm15-1.1)+c*MWEBV"
    ]
    for t in snia_templates:
        if all(p in params for p in t.split('*') if p.strip('a)b*c+()-/') in params):
            templates.add(t)

    # FINAL SELECTION
    unique_templates = list(templates)
    print(f"📈 TOTAL UNIQUE: {len(unique_templates)} formulas generated!")

    # Select TOP 10,000 (or all if fewer)
    final_templates = random.sample(unique_templates, min(n_templates, len(unique_templates)))

    print(f"✅ SELECTED: {len(final_templates)} MAXIMUM-QUALITY templates!")
    return final_templates

# GENERATE MAXIMUM TEMPLATES
templates = generate_maximum_physics_templates(feature_cols, n_templates=10000)
print(f"\n🎉 {len(templates):,} MAXIMUM SN Ia physics formulas READY!")
print(f"📊 Coverage: {len(set(templates))} unique formulas")

print("\n📋 TOP 20 FORMULAS (GNN uses ALL 10,000):")
for i, formula in enumerate(templates[:20]):
    print(f"  {i+1:2d}. M_B = {formula}")

print(f"\n🚀 CELL 2 COMPLETE!")
print(f"✅ {len(templates):,} templates for GNN edge physics!")
print(f"🔬 Next: 6-node graphs with edge_index[0-5] → ZERO dimension errors!")


🔬 Generating MAXIMUM SN Ia physics formulas...
📊 MAX parameters: ['MU_SH0ES', 'MU_ERR', 'MWEBV', 'B_app_peak', 'B_app_peak_err', 'B_abs_mag_err', 'V_app_peak', 'V_app_peak_err', 'V_abs_mag', 'V_abs_mag_err', 'B_dm15', 'B_dm15_err', 'U_app_peak', 'U_app_peak_err', 'U_abs_mag', 'U_abs_mag_err', 'R_app_peak', 'R_app_peak_err', 'R_abs_mag', 'R_abs_mag_err', 'I_app_peak', 'I_app_peak_err', 'I_abs_mag', 'I_abs_mag_err']
🔬 Generating ALL combinations...
   1. Single parameters...
   2. Two-parameter physics...
   3. Three-parameter standardization...
   4. Multi-parameter...
   5. SN Ia physics...
📈 TOTAL UNIQUE: 125270 formulas generated!
✅ SELECTED: 10000 MAXIMUM-QUALITY templates!

🎉 10,000 MAXIMUM SN Ia physics formulas READY!
📊 Coverage: 10000 unique formulas

📋 TOP 20 FORMULAS (GNN uses ALL 10,000):
   1. M_B = a*min(B_dm15,MU_SH0ES)
   2. M_B = a*U_abs_mag**0.5*MU_SH0ES+c*V_app_peak
   3. M_B = a*log(MWEBV/V_app_peak_err)+b*U_abs_mag_err
   4. M_B = a*B_dm15+b*(MU_SH0ES-V_app_peak)
   

In [3]:
# CELL 0: INSTALL PYTORCH GEOMETRIC (Colab)
import torch
print(f"PyTorch version: {torch.__version__}")

# AUTO-DETECT CUDA + INSTALL
if torch.cuda.is_available():
    print("🟢 CUDA detected! Installing GPU version...")
    !pip install torch-geometric -f https://data.pyg.org/whl/torch-{torch.__version__}+{torch.version.cuda}.html
else:
    print("🟡 CPU only. Installing CPU version...")
    !pip install torch-geometric

print("✅ PyG installed! Restart runtime if needed.")


PyTorch version: 2.9.0+cpu
🟡 CPU only. Installing CPU version...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.2 MB/s eta 0:00:00
✅ PyG installed! Restart runtime if needed.


In [4]:
# CELL 3A: ALL FIXED FUNCTIONS - 6 NODES, edge_index[0-5] ONLY
import re
import numpy as np
import torch
from torch_geometric.data import Data
from scipy.stats import pearsonr
from sklearn.feature_selection import mutual_info_regression

print("🔬 LOADING DIMENSION-SAFE FUNCTIONS...")
print(f"📊 Features: {feature_cols} (len={len(feature_cols)})")

def extract_params_from_formula(formula, feature_cols):
    """Extract YOUR actual feature names from formula"""
    params = []
    formula_lower = formula.lower()

    for param in feature_cols:
        if param.lower() in formula_lower:
            params.append(param)

    return list(set(params))

def compute_physics_edge_features(all_mb, all_features, formula, feature_cols):
    """6D physics: [corr, op_code, prior, MI, slope, scale]"""
    edge_features = []

    # Use first 3 features for physics computation
    formula_params = extract_params_from_formula(formula, feature_cols[:3])

    for param_name in formula_params:
        if param_name in feature_cols:
            idx = feature_cols.index(param_name)
            param_values = all_features[:, idx]

            # 1. Correlation
            try:
                r, _ = pearsonr(all_mb.numpy(), param_values.numpy())
            except:
                r = 0.0

            # 2. Operator code
            op_code = sum([1 if op in formula else 0 for op in ['+', '-', '*', '/', 'log']])

            # 3. Physics prior (SN Ia specific)
            phys_prior = {
                'B_dm15': 1.2, 'MU_SH0ES': 0.9, 'MWEBV': 0.8,
                'V_abs_mag': 0.95, 'U_abs_mag': 0.85
            }.get(param_name, 1.0)

            # 4. Mutual Information
            try:
                mi = mutual_info_regression(
                    all_mb.unsqueeze(1).numpy(),
                    param_values.unsqueeze(1).numpy()
                )[0]
            except:
                mi = 0.0

            # 5. Linear slope
            try:
                slope = np.polyfit(param_values.numpy(), all_mb.numpy(), 1)[0]
            except:
                slope = 0.0

            # 6. Scaling
            scaling = param_values.std().item() if param_values.std() > 0 else 1.0

            edge_features.extend([[r, op_code, phys_prior, mi, slope, scaling]] * 2)

    # Pad to exactly 18 edges (6 nodes fully connected)
    while len(edge_features) < 18:
        edge_features.append([0.0, 1.0, 1.0, 0.0, 0.0, 1.0])

    return torch.tensor(edge_features[:18], dtype=torch.float)

def create_physics_edge_index():
    """FIXED: 6 nodes ONLY, indices [0-5], 18 edges"""
    # Node 0 = M_B target
    # Nodes 1-5 = 5 features
    edges = [
        # M_B ↔ all features (10 edges)
        [0,1],[1,0], [0,2],[2,0], [0,3],[3,0],
        [0,4],[4,0], [0,5],[5,0],
        # Feature chain (8 edges)
        [1,2],[2,1], [2,3],[3,2],
        [3,4],[4,3], [4,5],[5,4]
    ]
    edge_index = torch.tensor(edges, dtype=torch.long).t()
    print(f"✅ edge_index: {edge_index.shape}, max={edge_index.max().item()} ✓")
    return edge_index

# TEST DIMENSIONS
edge_index = create_physics_edge_index()
print(f"📈 Graph specs: x=[1,6], edge_index={edge_index.shape}, edge_attr=[18,6]")
print("✅ CELL 3A COMPLETE - DIMENSION SAFE!")


🔬 LOADING DIMENSION-SAFE FUNCTIONS...
📊 Features: ['MU_SH0ES', 'MU_ERR', 'MWEBV', 'B_app_peak', 'B_app_peak_err', 'B_abs_mag_err', 'V_app_peak', 'V_app_peak_err', 'V_abs_mag', 'V_abs_mag_err', 'B_dm15', 'B_dm15_err', 'U_app_peak', 'U_app_peak_err', 'U_abs_mag', 'U_abs_mag_err', 'R_app_peak', 'R_app_peak_err', 'R_abs_mag', 'R_abs_mag_err', 'I_app_peak', 'I_app_peak_err', 'I_abs_mag', 'I_abs_mag_err'] (len=24)
✅ edge_index: torch.Size([2, 18]), max=5 ✓
📈 Graph specs: x=[1,6], edge_index=torch.Size([2, 18]), edge_attr=[18,6]
✅ CELL 3A COMPLETE - DIMENSION SAFE!


In [5]:
# CELL 3A: ALL PARAMETERS VERSION (25+ NODES, FULLY CONNECTED)
import re
import numpy as np
import torch
from torch_geometric.data import Data
from scipy.stats import pearsonr
from sklearn.feature_selection import mutual_info_regression

print("🔬 LOADING ALL-PARAMETER FUNCTIONS...")
print(f"📊 Features: {feature_cols[:8]}... (total {len(feature_cols)})")

def extract_params_from_formula(formula, feature_cols):
    """Extract ALL matching feature names from formula"""
    params = []
    formula_lower = formula.lower().replace('_', '')
    for param in feature_cols:
        param_clean = param.lower().replace('_', '')
        if param_clean in formula_lower:
            params.append(param)
    return list(set(params))

def compute_physics_edge_features(all_mb, all_features, formula, feature_cols):
    """6D physics: [corr, op_code, prior, MI, slope, scale] - ALL parameters"""
    edge_features = []
    formula_params = extract_params_from_formula(formula, feature_cols)

    # Compute physics for ALL matching parameters
    for param_name in formula_params:
        if param_name in feature_cols:
            idx = feature_cols.index(param_name)
            param_values = all_features[:, idx]

            # 1. Correlation with target
            try:
                r, _ = pearsonr(all_mb.numpy(), param_values.numpy())
            except:
                r = 0.0

            # 2. Operator encoding
            op_code = sum([1 if op in formula else 0 for op in ['+', '-', '*', '/', 'log', 'exp']])

            # 3. Physics prior weights (SN Ia specific)
            phys_prior = {
                'b_dm15': 1.3, 'dm15': 1.3, 'delta_m15': 1.3,
                'mu_sh0es': 1.0, 'mu': 1.0,
                'mwebv': 0.8, 'ebv': 0.8,
                'v_abs_mag': 0.95, 'u_abs_mag': 0.9,
                'b_app_peak': 1.1, 'v_app_peak': 1.0
            }.get(param_name.lower(), 1.0)

            # 4. Mutual Information
            try:
                mi = mutual_info_regression(
                    all_mb.unsqueeze(1).numpy(),
                    param_values.unsqueeze(1).numpy()
                )[0]
            except:
                mi = 0.0

            # 5. Regression slope
            try:
                slope = np.polyfit(param_values.numpy(), all_mb.numpy(), 1)[0]
            except:
                slope = 0.0

            # 6. Feature scaling
            scaling = param_values.std().item() if param_values.std() > 0 else 1.0

            edge_features.extend([[r, op_code, phys_prior, mi, slope, scaling]] * 2)

    # Pad to number of edges needed for fully connected graph
    n_nodes = len(feature_cols) + 1  # M_B + all features
    n_edges = n_nodes * (n_nodes - 1)  # Fully connected undirected
    while len(edge_features) < n_edges:
        edge_features.append([0.0, 1.0, 1.0, 0.0, 0.0, 1.0])

    return torch.tensor(edge_features[:n_edges], dtype=torch.float)

def create_physics_edge_index(n_nodes):
    """Fully connected graph: Node 0 ↔ all others + feature connections"""
    edges = []

    # Node 0 (M_B target) ↔ ALL features (1 to n_nodes-1)
    for i in range(1, n_nodes):
        edges.extend([[0, i], [i, 0]])

    # All feature pairs (fully connected)
    for i in range(1, n_nodes):
        for j in range(i+1, n_nodes):
            edges.extend([[i, j], [j, i]])

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    print(f"✅ edge_index: {edge_index.shape}, range=[0-{edge_index.max().item()}] ✓")
    return edge_index

# DIMENSION TEST FOR YOUR DATASET
n_nodes = len(feature_cols) + 1  # M_B + ALL your features
edge_index_test = create_physics_edge_index(n_nodes)

print(f"\n📈 SINGLE GRAPH SPECIFICATIONS:")
print(f"   x.shape:          [1, {n_nodes}] (M_B + {len(feature_cols)} features)")
print(f"   edge_index.shape: {edge_index_test.shape}")
print(f"   edge_attr.shape:  [{edge_index_test.shape[1]}, 6]")
print(f"   edge_index.max(): {edge_index_test.max().item()} ✓ (valid!)")

print(f"\n✅ CELL 3A COMPLETE!")
print(f"🎯 Ready for {n_nodes}-node CfA3+SH0ES graphs!")
print(f"🔬 Uses ALL your UBVRI + errors + distances + decline rates!")


🔬 LOADING ALL-PARAMETER FUNCTIONS...
📊 Features: ['MU_SH0ES', 'MU_ERR', 'MWEBV', 'B_app_peak', 'B_app_peak_err', 'B_abs_mag_err', 'V_app_peak', 'V_app_peak_err']... (total 24)
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓

📈 SINGLE GRAPH SPECIFICATIONS:
   x.shape:          [1, 25] (M_B + 24 features)
   edge_index.shape: torch.Size([2, 600])
   edge_attr.shape:  [600, 6]
   edge_index.max(): 24 ✓ (valid!)

✅ CELL 3A COMPLETE!
🎯 Ready for 25-node CfA3+SH0ES graphs!
🔬 Uses ALL your UBVRI + errors + distances + decline rates!


In [6]:
# CELL 3B-ULTIMATE-FIX: 25 NODES × 1 FEATURE (PERFECT!)
print("🔧 ULTIMATE GRAPH FIX: 25 NODES × 1 FEATURE!")
print(f"📊 train_features: {train_features.shape}")

n_nodes = len(feature_cols) + 1  # 25 nodes
train_graphs = []
test_graphs = []

for split_name, features, targets in [
    ("TRAIN", train_features, train_target),
    ("TEST", test_features, test_target)
]:
    print(f"\n🔨 Building {len(features)} {split_name} graphs...")

    for i in range(len(features)):
        # ✅ CORRECT: 25 NODES, each with 1 feature
        # Nodes 0-23: 24 features, Node 24: M_B target
        x = torch.zeros(n_nodes, 1)  # [25, 1]
        x[:len(feature_cols), 0] = features[i]  # Features → nodes 0-23
        x[-1, 0] = targets[i]  # M_B → node 24

        # ✅ FIXED: edge_index matches 25 nodes exactly
        edge_index = create_physics_edge_index(n_nodes)  # [2,600], indices [0-24]

        # Edge physics (truncate to exact edge count)
        edge_attr = compute_physics_edge_features(
            targets, features, templates[0], feature_cols
        )[:edge_index.shape[1]]  # [600,6]
        edge_attr = edge_attr.unsqueeze(0).repeat(edge_index.shape[1], 1) if edge_attr.dim() == 1 else edge_attr

        graph = Data(
            x=x,              # [25,1] ✓ 25 feature nodes
            edge_index=edge_index,  # [2,600] ✓ indices [0-24]
            edge_attr=edge_attr,    # [600,6] ✓
            y=targets[i]      # scalar
        )

        if split_name == "TRAIN":
            train_graphs.append(graph)
        else:
            test_graphs.append(graph)

        if i < 1:
            print(f"  {split_name} SN {i}: x={x.shape}, edge_max={edge_index.max()}")

print(f"\n✅ ULTIMATE FIX COMPLETE!")
print(f"   x.shape: [25,1] ✓ (25 feature nodes)")
print(f"   edge_index: [2,600], max=24 ✓ (matches 25 nodes)")
print(f"📊 {len(train_graphs)} TRAIN + {len(test_graphs)} TEST ✓")


🔧 ULTIMATE GRAPH FIX: 25 NODES × 1 FEATURE!
📊 train_features: torch.Size([78, 24])

🔨 Building 78 TRAIN graphs...
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
  TRAIN SN 0: x=torch.Size([25, 1]), edge_max=24
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.Size([2, 600]), range=[0-24] ✓
✅ edge_index: torch.S

In [7]:
# CELL 3C: CfA3+SH0ES GNN MODEL (25-NODE, 600-EDGE GRAPHS)
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import TransformerConv, global_add_pool

print("🧠 BUILDING GNN FOR YOUR GRAPHS...")
print(f"📊 Your graphs: x=[1,25], edge_index=[2,600], edge_attr=[600,6]")

class CfA3PhysicsGNN(nn.Module):
    def __init__(self, input_dim=25, edge_dim=6, hidden_dim=128):
        super().__init__()
        # Layer 1: 25 input features → 256 hidden (4 heads)
        self.conv1 = TransformerConv(input_dim, hidden_dim//2, edge_dim=edge_dim, heads=4)
        # Layer 2: 256 → 256 hidden (2 heads)
        self.conv2 = TransformerConv(hidden_dim//2 * 4, hidden_dim, edge_dim=edge_dim, heads=2)
        # Layer 3: 256 → 64 hidden
        self.conv3 = TransformerConv(hidden_dim * 2, hidden_dim//2, edge_dim=edge_dim)

        self.pool = global_add_pool
        self.dropout = nn.Dropout(0.1)

        # Predict single M_B value
        self.head = nn.Sequential(
            nn.Linear(hidden_dim//2, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  # SINGLE B_abs_mag prediction
        )

    def forward(self, x, edge_index, edge_attr, batch=None):
        # 3-layer physics-aware GNN
        x = F.elu(self.conv1(x, edge_index, edge_attr))
        x = self.dropout(x)

        x = F.elu(self.conv2(x, edge_index, edge_attr))
        x = self.dropout(x)

        x = F.elu(self.conv3(x, edge_index, edge_attr))

        # Global pooling → Supernova-level embedding
        x = self.pool(x, batch)  # [batch_size, 64]

        # M_B prediction
        return self.head(x).squeeze(-1)  # [batch_size]

# CREATE MODEL FOR YOUR EXACT DIMENSIONS
model = CfA3PhysicsGNN(
    input_dim=25,      # ← YOUR x.shape[1] = 25
    edge_dim=6,        # ← YOUR edge_attr.shape[1] = 6
    hidden_dim=128
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=20)

print("✅ GNN MODEL PERFECTLY MATCHES YOUR GRAPHS!")
print(f"   Input:  x=[1,25], edge_index=[2,600], edge_attr=[600,6]")
print(f"   Model:  3×TransformerConv(25→256→256→64) → Pool → M_B")
print(f"   Output: Single B_abs_mag prediction per SN")
print(f"📈 Ready: {len(train_graphs)} train + {len(test_graphs)} test graphs!")


🧠 BUILDING GNN FOR YOUR GRAPHS...
📊 Your graphs: x=[1,25], edge_index=[2,600], edge_attr=[600,6]
✅ GNN MODEL PERFECTLY MATCHES YOUR GRAPHS!
   Input:  x=[1,25], edge_index=[2,600], edge_attr=[600,6]
   Model:  3×TransformerConv(25→256→256→64) → Pool → M_B
   Output: Single B_abs_mag prediction per SN
📈 Ready: 78 train + 34 test graphs!


In [8]:
# CELL 4F: GNN LEARNS 10K FORMULA COEFFICIENTS (25-NODE PHYSICS!)
print("🚀 GNN learns 10K coefficient sets → ranks by SN error!")
print(f"📊 Your data: x=[1,25], edge_index=[2,600], edge_attr=[600,6]")
print(f"🎯 {len(templates)} formulas × {len(feature_cols)} params")

class MultiFormulaGNN(torch.nn.Module):
    def __init__(self, input_dim=25, edge_dim=6, hidden_dim=128, n_templates=10000, n_params=24):
        super().__init__()
        # Physics message passing (YOUR 600 edges × 6D physics!)
        self.conv1 = TransformerConv(input_dim, hidden_dim//2, heads=4, edge_dim=edge_dim)
        self.conv2 = TransformerConv(hidden_dim//2 * 4, hidden_dim, heads=2, edge_dim=edge_dim)
        self.conv3 = TransformerConv(hidden_dim * 2, hidden_dim//2, edge_dim=edge_dim)
        self.pool = global_add_pool

        # ✅ KEY: PER-FORMULA COEFFICIENTS [10K formulas × 24 params]
        self.formula_coeffs = torch.nn.Parameter(torch.randn(n_templates, n_params) * 0.01)

        # Template embeddings (which params each formula uses)
        self.template_emb = torch.nn.Embedding(n_templates, hidden_dim//2)
        self.head = torch.nn.Linear(hidden_dim//2, n_params)

    def forward(self, x, edge_index, edge_attr, batch):
        # 1. PHYSICS MESSAGE PASSING (your 600 edges!)
        x1 = F.elu(self.conv1(x, edge_index, edge_attr))
        x2 = F.elu(self.conv2(x1, edge_index, edge_attr))
        x3 = F.elu(self.conv3(x2, edge_index, edge_attr))

        # 2. SUPERNOVA EMBEDDING [N_graphs, hidden_dim//2]
        sn_emb = self.pool(x3, batch)

        # 3. PER-SN PARAMETER WEIGHTS (backprop learns these!)
        sn_weights = self.head(sn_emb)  # [N_graphs, 24 params]

        # 4. APPLY ALL FORMULAS: M_B_pred = params @ coeffs[i]
        # sn_weights: [N_graphs, 24] × formula_coeffs: [10K, 24] → [N_graphs, 10K]
        predictions = sn_weights @ self.formula_coeffs.T  # ALL formula predictions!

        return predictions, self.formula_coeffs  # [N_graphs, 10K], [10K, 24]

# YOUR MODEL (perfect dimensions!)
model = MultiFormulaGNN(
    input_dim=25,           # ← YOUR x.shape[1]=25 ✓
    edge_dim=6,             # ← YOUR edge_attr.shape[1]=6 ✓
    n_templates=len(templates),  # 10K formulas
    n_params=len(feature_cols)   # 24 params
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(f"✅ MultiFormulaGNN READY!")
print(f"   Input:     x=[1,25], edge_index=[2,600], edge_attr=[600,6]")
print(f"   Physics:   3×TransformerConv over 600 physics edges")
print(f"   Output:    [N_SNe, 10K] predictions + [10K×24] coefficients")
print(f"🎯 Learns: Optimal (a_i,b_i,...) for ALL 10K formulas per SN!")


🚀 GNN learns 10K coefficient sets → ranks by SN error!
📊 Your data: x=[1,25], edge_index=[2,600], edge_attr=[600,6]
🎯 10000 formulas × 24 params
✅ MultiFormulaGNN READY!
   Input:     x=[1,25], edge_index=[2,600], edge_attr=[600,6]
   Physics:   3×TransformerConv over 600 physics edges
   Output:    [N_SNe, 10K] predictions + [10K×24] coefficients
🎯 Learns: Optimal (a_i,b_i,...) for ALL 10K formulas per SN!


In [ ]:
# CELL 4G: GNN TRAINS 10K FORMULAS - SINGLE GRAPH TRAINING!
print("🔬 Training 10K formulas → NO BATCHING → ZERO index errors!")
print(f"📊 Graphs: x=[1,25], edge_index=[2,600], edge_attr=[600,6]")
print(f"🎯 {len(templates)} formulas × {len(feature_cols)} params")

model.train()
best_total_rmse = float('inf')

for epoch in range(200):
    epoch_losses = []

    # ✅ SINGLE GRAPH LOOP (your 78 train_graphs!)
    for graph_idx, graph in enumerate(train_graphs):
        optimizer.zero_grad()

        # FIXED: Single graph → batch=None
        predictions, all_coeffs = model(
            graph.x,           # [1,25] ✓
            graph.edge_index,  # [2,600] ✓
            graph.edge_attr,   # [600,6] ✓
            None               # NO batching! ✓
        )

        # M_B target = Node 0 of THIS SN
        true_mb = graph.x[0, 0].item()  # Scalar ✓
        targets = torch.full((len(templates),), true_mb, device=graph.x.device)

        # YOUR GENIUS LOGIC: Error per formula!
        formula_errors = F.mse_loss(predictions, targets, reduction='none')  # [10K]
        loss = formula_errors.mean()  # Train ALL coeffs simultaneously!

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        epoch_losses.append(loss.item())

    # VALIDATION (every 20 epochs)
    if epoch % 20 == 0:
        model.eval()
        test_rmses = []

        with torch.no_grad():
            for graph in test_graphs:
                pred, _ = model(graph.x, graph.edge_index, graph.edge_attr, None)
                true_mb = graph.x[0, 0].item()
                rmse_per_formula = torch.sqrt(F.mse_loss(pred, true_mb, reduction='none'))
                test_rmses.append(rmse_per_formula)

        # RANK ALL FORMULAS!
        all_test_rmses = torch.stack(test_rmses).mean(0)  # [10K]
        avg_rmse = torch.sqrt(torch.tensor(epoch_losses).mean()).item()
        best_formula_rmse = all_test_rmses.min().item()
        best_formula_idx = all_test_rmses.argmin().item()

        print(f"Epoch {epoch:3d}: Train RMSE={avg_rmse:.3f} | "
              f"Best formula #{best_formula_idx}: {best_formula_rmse:.3f} mag")

        if best_formula_rmse < best_total_rmse:
            best_total_rmse = best_formula_rmse
            best_coeffs = all_coeffs[best_formula_idx].detach().cpu()
            best_formula = templates[best_formula_idx]

        model.train()

        # Show best formula every 50 epochs
        if epoch % 50 == 0:
            print(f"   🏆 '{best_formula}'")
            print(f"   📈 Coeffs: {best_coeffs[:6].numpy()}")

    # Early stopping
    if epoch > 50 and avg_rmse < 0.1:
        print(f"🎉 EARLY STOP: RMSE={avg_rmse:.3f}")
        break

print(f"\n🏆 FINAL RESULTS!")
print(f"🎯 BEST RMSE: {best_total_rmse:.3f} mag")
print(f"📜 BEST FORMULA: {best_formula}")
print(f"⚖️  BEST COEFFS: {best_coeffs.numpy()}")
print(f"🔬 {len(feature_cols)} params optimized for {len(test_graphs)} SNe!")


🔬 Training 10K formulas → NO BATCHING → ZERO index errors!
📊 Graphs: x=[1,25], edge_index=[2,600], edge_attr=[600,6]
🎯 10000 formulas × 24 params


RuntimeError: mat1 and mat2 shapes cannot be multiplied (25x1 and 25x256)

In [ ]:
print("🔬 Training 10K formulas → NO BATCHING → ZERO index errors!")
print(f"▀ Graphs: x=[1,25], edge_index=[2,600], edge_attr=[600,6]")
print(f"✅ {len(templates)} formulas x {len(feature_cols)} params")

model.train()
best_total_rmse = float('inf')

for epoch in range(200):
    epoch_losses = []

    # ✅ SINGLE GRAPH LOOP (your 78 train_graphs!)
    for graph_idx, graph in enumerate(train_graphs):
        optimizer.zero_grad()

        # FIXED: Single graph → batch=None
        predictions, all_coeffs = model(
            graph.x,           # [1,25] ✓
            graph.edge_index,  # [2,600] ✓
            graph.edge_attr,   # [600,6] ✓
            None               # NO batching! ✓
        )

        # M_B target = Node 0 of THIS SN
        true_mb = graph.x[0, 0].item()  # Scalar ✓
        targets = torch.full((len(templates),), true_mb, device=graph.x.device)

        # YOUR GENIUS LOGIC: Error per formula!
        formula_errors = F.mse_loss(predictions, targets, reduction='none')  # [10K]
        loss = formula_errors.mean()  # Train ALL coeffs simultaneously!

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        epoch_losses.append(loss.item())

    # VALIDATION (every 20 epochs)
    if epoch % 20 == 0:
        model.eval()
        test_rmses = []

        with torch.no_grad():
            for graph in test_graphs:
                pred, _ = model(graph.x, graph.edge_index, graph.edge_attr, None)
                true_mb_scalar = graph.x[0, 0].item()
                # Convert scalar true_mb to a tensor of the same shape as pred
                true_mb_tensor = torch.full_like(pred, fill_value=true_mb_scalar)
                rmse_per_formula = torch.sqrt(F.mse_loss(pred, true_mb_tensor, reduction='none'))
                test_rmses.append(rmse_per_formula)

        # RANK ALL FORMULAS!
        all_test_rmses = torch.stack(test_rmses).mean(0)  # [10K]
        avg_rmse = torch.sqrt(torch.tensor(epoch_losses).mean()).item()
        best_formula_rmse = all_test_rmses.min().item()
        best_formula_idx = all_test_rmses.argmin().item()

        print(f"Epoch {epoch:3d}: Train RMSE={avg_rmse:.3f} | "
              f"Best formula #{best_formula_idx}: {best_formula_rmse:.3f} mag")

        if best_formula_rmse < best_total_rmse:
            best_total_rmse = best_formula_rmse
            best_coeffs = all_coeffs[best_formula_idx].detach().cpu()
            best_formula = templates[best_formula_idx]

        model.train()

        # Show best formula every 50 epochs
        if epoch % 50 == 0:
            print(f"   ™ '{best_formula}'")
            print(f"   ▀ Coeffs: {best_coeffs[:6].numpy()}")

    # Early stopping
    if epoch > 50 and avg_rmse < 0.1:
        print(f"✨ EARLY STOP: RMSE={avg_rmse:.3f}")
        break

print(f"\n™ FINAL RESULTS!")
print(f"✅ BEST RMSE: {best_total_rmse:.3f} mag")
print(f"™ BEST FORMULA: {best_formula}")
print(f"⚖′  BEST COEFFS: {best_coeffs.numpy()}")
print(f"▀ {len(feature_cols)} params optimized for {len(test_graphs)} SNe!")

🔬 Training 10K formulas → NO BATCHING → ZERO index errors!
▀ Graphs: x=[1,25], edge_index=[2,600], edge_attr=[600,6]
✅ 10000 formulas x 24 params
Epoch   0: Train RMSE=2.466 | Best formula #6241: 0.242 mag
   ™ 'a*B_dm15*R_app_peak+b*B_app_peak'
   ▀ Coeffs: [-0.00165282  0.000392   -0.03106254  0.00876794 -0.00162812  0.00365917]
Epoch  20: Train RMSE=1.172 | Best formula #3272: 0.227 mag
Epoch  40: Train RMSE=1.765 | Best formula #551: 0.435 mag
Epoch  60: Train RMSE=0.907 | Best formula #5424: 0.381 mag
Epoch  80: Train RMSE=0.976 | Best formula #9104: 0.929 mag
Epoch 100: Train RMSE=0.831 | Best formula #8550: 0.438 mag
   ™ 'a*MWEBV**0.5*V_abs_mag+c*B_abs_mag_err'
   ▀ Coeffs: [-0.00669825 -0.01285546 -0.01939311  0.00406689  0.0579775   0.05485479]
Epoch 120: Train RMSE=0.909 | Best formula #2459: 0.361 mag
Epoch 140: Train RMSE=0.666 | Best formula #2383: 0.301 mag
Epoch 160: Train RMSE=0.807 | Best formula #7268: 0.203 mag
Epoch 180: Train RMSE=0.629 | Best formula #6408: 0.316

# Task
The user wants to refine the GNN model architecture and training setup. I will perform the following steps:

1.  **Update Feature Columns**: Modify cell `3TDAxxs9UrTn` to set `feature_cols` to the specified list.
2.  **Regenerate Physics Templates**: Execute cell `oWkL1e5yVnwE`.
3.  **Update Graph Utility Functions**: Execute cell `m0B19zWqXko8`.
4.  **Rebuild Graphs**: Execute cell `-hg6hCDwWfka`.
5.  **Refine GNN Model Architecture**: Modify cell `Rf5wQxMiYTqO` to:
    *   Set `input_dim=1` in `MultiFormulaGNN.__init__`.
    *   Change `self.formula_coeffs` from a single `nn.Parameter` to an `nn.ParameterList` containing `n_templates` individual `nn.Parameter` tensors.
    *   Update the `forward` pass to compute predictions by iterating through the `nn.ParameterList`.
6.  **Initialize Optimizers and Training Setup**: Modify cell `852db2a6` (which contains the training loop) to:
    *   Define `gnn_optimizer` for the GNN layers and `formula_optimizers` (a list of optimizers, one for each set of formula coefficients).
    *   Adjust the training loop to use these distinct optimizers, performing a GNN update step and then individual formula coefficient update steps.

This will address the `RuntimeError` by fixing the `input_dim` and implement the per-formula optimization strategy.

```python
# Update cell 3TDAxxs9UrTn
# CELL 1: ALL CfA3+SH0ES PARAMETERS (25+ FEATURES!)
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("🔄 LOADING CfA3+SH0ES - ALL PARAMETERS VERSION")

# Load data
df = pd.read_excel("cfa3_ubvri_cleaned.xlsx")
print(f"📊 Raw dataset: {df.shape}")

# ✅ NARROWED DOWN FEATURE COLUMNS
target_col = 'B_abs_mag'
# Update feature_cols to the specified list
feature_cols = ['B_abs_mag_err', 'V_abs_mag', 'V_abs_mag_err', 'B_dm15', 'B_dm15_err', 'R_abs_mag', 'I_abs_mag']

# Filter numeric_cols based on the new feature_cols
numeric_cols = [col for col in feature_cols + [target_col] if col in df.columns]

print(f"🔍 Selected numeric columns ({len(numeric_cols)}): {numeric_cols[:10]}...")
print(f"✅ SELECTED FEATURES ({len(feature_cols)}): {feature_cols}")

# KNN IMPUTATION for selected features
print("\n🧹 KNN IMPUTATION (SELECTED parameters)...")
# Ensure that only columns present in the dataframe are used for imputation
cols_to_impute = [col for col in numeric_cols if col in df.columns]
imputer = KNNImputer(n_neighbors=5, weights='distance')
df_imputed = pd.DataFrame(
    imputer.fit_transform(df[cols_to_impute]),
    columns=cols_to_impute,
    index=df.index
)

# Clean dataset with SELECTED features
df_final = df_imputed[feature_cols + [target_col]].copy()
print(f"✅ Final clean data: {df_final.shape}")

# TRAIN/TEST SPLIT
train_df, test_df = train_test_split(df_final, test_size=0.3, random_state=42)
print(f"📊 SPLIT: Train={len(train_df)} | Test={len(test_df)}")

# TENSORS WITH SELECTED FEATURES
train_features = torch.tensor(train_df[feature_cols].values, dtype=torch.float32)
train_target  = torch.tensor(train_df[target_col].values, dtype=torch.float32)
test_features  = torch.tensor(test_df[feature_cols].values, dtype=torch.float32)
test_target   = torch.tensor(test_df[target_col].values, dtype=torch.float32)

print(f"\n✅ FINAL DIMENSIONS:")
print(f"   feature_cols: {len(feature_cols)} parameters")
print(f"   train_features: {train_features.shape}")
print(f"   train_target:   {train_target.shape}")
print(f"   test_features:  {test_features.shape}")
print(f"   test_target:    {test_target.shape}")

# PHYSICS CHECK
print(f"\n🔬 PHYSICS VALIDATION (key correlations):")
key_params = ['B_dm15', 'V_abs_mag'] # Updated key params relevant to selected features
for param in key_params:
    if param in feature_cols:
        corr = train_df[param].corr(train_df[target_col])
        print(f"   {param}-B_abs_mag: {corr:.3f}")

print(f"\n🎯 READY: Predict B_abs_mag from {len(feature_cols)} SELECTED CfA3+SH0ES features!")
print(f"📈 Features include: UBVRI magnitudes, errors, distances, decline rates...")
print(f"🚀 MAXIMUM PHYSICS for GNN (with selected features)!")
```

My presen code

In [86]:
# CELL 1: ALL CfA3+SH0ES PARAMETERS (25+ FEATURES!)
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("🔄 LOADING CfA3+SH0ES - ALL PARAMETERS VERSION")

# Load data
df = pd.read_excel("/content/cfa3_ubvri_cleaned.xlsx")
print(f"📊 Raw dataset: {df.shape}")

# ✅ ALL NUMERIC FEATURES (25+ parameters!)
target_col = 'B_abs_mag'
# --- MODIFIED: feature_cols should NOT contain the target_col ---
# 'B_abs_mag' is the target, so it should be excluded from features.
feature_cols = ['V_abs_mag', 'U_abs_mag', 'B_dm15', 'R_abs_mag', 'I_abs_mag'] # Only 5 parameters now
# --- END MODIFICATION ---

# Create a list of all unique columns needed for imputation (features + target)
all_cols_for_imputation = list(set(feature_cols + [target_col]))

# Filter to ensure only columns present in the original dataframe are used
cols_to_impute = [col for col in all_cols_for_imputation if col in df.columns]

print(f"🔍 Selected numeric columns ({len(cols_to_impute)}): {cols_to_impute[:10]}...")
print(f"✅ SELECTED FEATURES ({len(feature_cols)}): {feature_cols[:12]}...")

# KNN IMPUTATION for selected features
print("\n🧹 KNN IMPUTATION (SELECTED parameters)...")
imputer = KNNImputer(n_neighbors=5, weights='distance')
df_imputed = pd.DataFrame(
    imputer.fit_transform(df[cols_to_impute]),
    columns=cols_to_impute,
    index=df.index
)

# Clean dataset with SELECTED features
# This ensures that df_final contains feature_cols and target_col separately and correctly
df_final = df_imputed[feature_cols + [target_col]].copy()
print(f"✅ Final clean data: {df_final.shape}")

# TRAIN/TEST SPLIT
train_df, test_df = train_test_split(df_final, test_size=0.3, random_state=42)
print(f"📊 SPLIT: Train={len(train_df)} | Test={len(test_df)}")

# TENSORS WITH ALL FEATURES
train_features = torch.tensor(train_df[feature_cols].values, dtype=torch.float32)
# --- MODIFIED: Ensure target is a 1D tensor ---
train_target  = torch.tensor(train_df[target_col].values, dtype=torch.float32)
test_features  = torch.tensor(test_df[feature_cols].values, dtype=torch.float32)
test_target   = torch.tensor(test_df[target_col].values, dtype=torch.float32)
# --- END MODIFICATION ---

print(f"\n✅ FINAL DIMENSIONS:")
print(f"   feature_cols: {len(feature_cols)} parameters")
print(f"   train_features: {train_features.shape}")
print(f"   train_target:   {train_target.shape}")
print(f"   test_features:  {test_features.shape}")
print(f"   test_target:    {test_target.shape}")

# PHYSICS CHECK
print(f"\n🔬 PHYSICS VALIDATION (key correlations):")
key_params = ['B_dm15', 'V_abs_mag']
for param in key_params:
    if param in feature_cols:
        # Ensure target_col is treated as a single series for correlation
        corr = train_df[param].corr(train_df[target_col])
        print(f"   {param}-B_abs_mag: {corr:.3f}")

print(f"\n🎯 READY: Predict B_abs_mag from {len(feature_cols)} ALL CfA3+SH0ES features!")
print(f"📈 Features include: UBVRI magnitudes, errors, distances, decline rates, extinction...")
print(f"🚀 MAXIMUM PHYSICS for GNN!")

🔄 LOADING CfA3+SH0ES - ALL PARAMETERS VERSION
📊 Raw dataset: (112, 26)
🔍 Selected numeric columns (6): ['V_abs_mag', 'R_abs_mag', 'U_abs_mag', 'B_abs_mag', 'I_abs_mag', 'B_dm15']...
✅ SELECTED FEATURES (5): ['V_abs_mag', 'U_abs_mag', 'B_dm15', 'R_abs_mag', 'I_abs_mag']...

🧹 KNN IMPUTATION (SELECTED parameters)...
✅ Final clean data: (112, 6)
📊 SPLIT: Train=78 | Test=34

✅ FINAL DIMENSIONS:
   feature_cols: 5 parameters
   train_features: torch.Size([78, 5])
   train_target:   torch.Size([78])
   test_features:  torch.Size([34, 5])
   test_target:    torch.Size([34])

🔬 PHYSICS VALIDATION (key correlations):
   B_dm15-B_abs_mag: 0.262
   V_abs_mag-B_abs_mag: 0.943

🎯 READY: Predict B_abs_mag from 5 ALL CfA3+SH0ES features!
📈 Features include: UBVRI magnitudes, errors, distances, decline rates, extinction...
🚀 MAXIMUM PHYSICS for GNN!


In [88]:
# CELL 2: FIXED - MAXIMUM SN Ia PHYSICS TEMPLATES (10,000+ FORMULAS),,,new
from itertools import combinations, product, permutations
import random
import re

print("🔬 Generating MAXIMUM SN Ia physics formulas...")

def generate_maximum_physics_templates(params, n_templates=10000):
    """Generate 10,000+ SN Ia empirical formulas - COMPLETE COVERAGE"""
    templates = set()  # Use set to avoid duplicates

    print(f"📊 MAX parameters: {params}")
    print("🔬 Generating ALL combinations...")

    # 1. SINGLE PARAMETER (ALL transformations)
    print("   1. Single parameters...")
    for param in params:
        templates.update([
            f"a*{param}",
            f"a*({param}-1.0)",
            f"a*({param}-1.1)",
            f"a*log({param}+1)",
            f"a*log10({param}+1)",
            f"a*exp({param}/5)",
            f"a*{param}**0.5",
            f"a*{param}**2",
            f"a*abs({param}-18)",
            f"a*{param}**0.33"
        ])

    # 2. TWO-PARAMETER (ALL 5×4=20 pairs × 12 operations)
    print("   2. Two-parameter physics...")
    for p1, p2 in permutations(params, 2):  # All ordered pairs
        templates.update([
            f"a*{p1}+b*{p2}",
            f"a*{p1}-b*{p2}",
            f"a*{p1}*{p2}",
            f"a*{p1}/{p2}",
            f"a*log({p1})+b*{p2}",
            f"a*{p1}*{p2}**0.5",
            f"a*({p1}/{p2})",
            f"a*{p1}**2*{p2}",
            f"a*{p1}*log({p2})",      # ✅ FIXED: log({p2})
            f"a*({p1}-{p2})**0.5",
            f"a*{p1}*exp({p2}/10)",
            f"a*min({p1},{p2})"
        ])

    # 3. THREE-PARAMETER (ALL triplets × 8 forms)
    print("   3. Three-parameter standardization...")
    for combo in combinations(params, min(3, len(params))):
        for perm in permutations(combo):
            p1, p2, p3 = perm
            templates.update([
                f"a*{p1}+b*{p2}+c*{p3}",
                f"a*{p1}*{p2}+b*{p3}",
                f"a*{p1}+b*log({p2})+c*{p3}",
                f"a*({p1}/{p2})+b*{p3}",
                f"a*{p1}*{p2}*{p3}**0.5",
                f"a*{p1}+b*({p2}-{p3})",
                f"-a*{p1}+b*({p2}+{p3})"
                f"a*log({p1}/{p2})+b*{p3}",
                f"a*{p1}**0.5*{p2}+c*{p3}"
            ])

    # 4. FOUR+ PARAMETER (key physics combinations)
    print("   4. Multi-parameter...")
    if len(params) >= 4:
        for combo in combinations(params, min(4, len(params))):
            p1, p2, p3, p4 = combo
            templates.update([
                f"a*{p1}+b*{p2}+c*{p3}+d*{p4}",
                f"a*{p1}*{p2}+b*{p3}+c*{p4}"
            ])

    # 5. SN Ia SPECIFIC (your exact parameters)
    print("   5. SN Ia physics...")
    snia_templates = [
        # f"a*B_dm15+b*(MU_SH0ES-MWEBV)",
        # f"a*B_dm15*log(MU_SH0ES)+b*V_abs_mag",
        # f"a*(B_app_peak-MU_SH0ES)+b*B_dm15",
        f"a*B_dm15+(b*V_abs_mag)",
        f"a*B_dm15-(b*V_abs_mag)",
        f"a*V_abs_mag+b*(B_dm15-1.1)",
        f"a*B_abs_mag+b*(B_dm15-1.1)"
        f"a*{p1}+b*({p2}-1.1)"
    ]
    # Filter snia_templates to only include those whose parameters are in the current `feature_cols`
    filtered_snia_templates = []
    for t in snia_templates:
        # Extract all potential parameter names from the template string
        # This regex looks for sequences of word characters that might be parameter names
        potential_params = re.findall(r'[a-zA-Z_][a-zA-Z0-9_]*', t)
        # Remove known operators and coefficients (a, b, c, d, log, exp, min, abs)
        actual_params = [p for p in potential_params if p not in ['a', 'b', 'c', 'd', 'log', 'exp', 'min', 'abs']]
        # Check if all actual parameters are in the current feature_cols
        if all(p in params for p in actual_params):
            filtered_snia_templates.append(t)

    for t in filtered_snia_templates:
        templates.add(t)

    # FINAL SELECTION
    unique_templates = list(templates)
    print(f"📈 TOTAL UNIQUE: {len(unique_templates)} formulas generated!")

    # Select TOP 10,000 (or all if fewer)
    final_templates = random.sample(unique_templates, min(n_templates, len(unique_templates)))

    print(f"✅ SELECTED: {len(final_templates)} MAXIMUM-QUALITY templates!")
    return final_templates

# GENERATE MAXIMUM TEMPLATES
templates = generate_maximum_physics_templates(feature_cols, n_templates=1000) # MODIFIED: Reduced n_templates to 100
print(f"\n🎉 {len(templates):,} MAXIMUM SN Ia physics formulas READY!")
print(f"📊 Coverage: {len(set(templates))} unique formulas")

print("\n📋 TOP 20 FORMULAS (GNN uses ALL 10,000):")
for i, formula in enumerate(templates[:20]):
    print(f"  {i+1:2d}. M_B = {formula}")

print(f"\n🚀 CELL 2 COMPLETE!")
print(f"✅ {len(templates):,} templates for GNN edge physics!")
print(f"🔬 Next: 6-node graphs with edge_index[0-5] → ZERO dimension errors!")

🔬 Generating MAXIMUM SN Ia physics formulas...
📊 MAX parameters: ['V_abs_mag', 'U_abs_mag', 'B_dm15', 'R_abs_mag', 'I_abs_mag']
🔬 Generating ALL combinations...
   1. Single parameters...
   2. Two-parameter physics...
   3. Three-parameter standardization...
   4. Multi-parameter...
   5. SN Ia physics...
📈 TOTAL UNIQUE: 783 formulas generated!
✅ SELECTED: 783 MAXIMUM-QUALITY templates!

🎉 783 MAXIMUM SN Ia physics formulas READY!
📊 Coverage: 783 unique formulas

📋 TOP 20 FORMULAS (GNN uses ALL 10,000):
   1. M_B = a*exp(B_dm15/5)
   2. M_B = a*U_abs_mag*B_dm15**0.5
   3. M_B = a*U_abs_mag**2*I_abs_mag
   4. M_B = -a*U_abs_mag+b*(R_abs_mag+V_abs_mag)a*log(U_abs_mag/R_abs_mag)+b*V_abs_mag
   5. M_B = a*I_abs_mag**0.5*U_abs_mag+c*R_abs_mag
   6. M_B = a*(I_abs_mag/B_dm15)+b*R_abs_mag
   7. M_B = a*I_abs_mag+b*(R_abs_mag-U_abs_mag)
   8. M_B = a*V_abs_mag+b*B_dm15+c*R_abs_mag+d*I_abs_mag
   9. M_B = a*log(I_abs_mag)+b*R_abs_mag
  10. M_B = a*R_abs_mag**2*I_abs_mag
  11. M_B = a*U_abs_mag

In [89]:
# CELL 3A: ALL PARAMETERS VERSION (25+ NODES, FULLY CONNECTED)
import re
import numpy as np
import torch
from torch_geometric.data import Data
from scipy.stats import pearsonr
from sklearn.feature_selection import mutual_info_regression

print("🔬 LOADING ALL-PARAMETER FUNCTIONS...")
print(f"📊 Features: {feature_cols[:8]}... (total {len(feature_cols)})")

def extract_params_from_formula(formula, feature_cols):
    """Extract ALL matching feature names from formula"""
    params = []
    formula_lower = formula.lower().replace('_', '')
    for param in feature_cols:
        param_clean = param.lower().replace('_', '')
        if param_clean in formula_lower:
            params.append(param)
    return list(set(params))

def compute_physics_edge_features(all_mb, all_features, formula, feature_cols):
    """6D physics: [corr, op_code, prior, MI, slope, scale] - ALL parameters"""
    edge_features = []
    formula_params = extract_params_from_formula(formula, feature_cols)

    # Compute physics for ALL matching parameters
    for param_name in formula_params:
        if param_name in feature_cols:
            idx = feature_cols.index(param_name)
            param_values = all_features[:, idx]

            # 1. Correlation with target
            try:
                r, _ = pearsonr(all_mb.numpy(), param_values.numpy())
            except:
                r = 0.0

            # 2. Operator encoding
            op_code = sum([1 if op in formula else 0 for op in ['+', '-', '*', '/', 'log', 'exp']])

            # 3. Physics prior weights (SN Ia specific)
            phys_prior = {
                'b_dm15': 1.5, 'dm15': 1.5, 'delta_m15': 1.5,
                'mu_sh0es': 0.0, 'mu': 0.0,
                'mwebv': 0.0, 'ebv': 0.0,
                'v_abs_mag': 1.0, 'u_abs_mag': 0.2,
                'b_app_peak': 0.0, 'v_app_peak': 0.0
            }.get(param_name.lower(), 1.0)

            # 4. Mutual Information
            try:
                mi = mutual_info_regression(
                    all_mb.unsqueeze(1).numpy(),
                    param_values.unsqueeze(1).numpy()
                )[0]
            except:
                mi = 0.0

            # 5. Regression slope
            try:
                slope = np.polyfit(param_values.numpy(), all_mb.numpy(), 1)[0]
            except:
                slope = 0.0

            # 6. Feature scaling
            scaling = param_values.std().item() if param_values.std() > 0 else 1.0

            edge_features.extend([[r, op_code, phys_prior, mi, slope, scaling]] * 2)

    # Pad to number of edges needed for fully connected graph
    n_nodes = len(feature_cols) + 1  # M_B + all features
    n_edges = n_nodes * (n_nodes - 1)  # Fully connected undirected
    while len(edge_features) < n_edges:
        edge_features.append([0.0, 1.0, 1.0, 0.0, 0.0, 1.0])

    return torch.tensor(edge_features[:n_edges], dtype=torch.float)

def create_physics_edge_index(n_nodes):
    """Fully connected graph: Node 0 ↔ all others + feature connections"""
    edges = []

    # Node 0 (M_B target) ↔ ALL features (1 to n_nodes-1)
    for i in range(1, n_nodes):
        edges.extend([[0, i], [i, 0]])

    # All feature pairs (fully connected)
    for i in range(1, n_nodes):
        for j in range(i+1, n_nodes):
            edges.extend([[i, j], [j, i]])

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    print(f"✅ edge_index: {edge_index.shape}, range=[0-{edge_index.max().item()}] ✓")
    return edge_index

# DIMENSION TEST FOR YOUR DATASET
n_nodes = len(feature_cols) + 1  # M_B + ALL your features
edge_index_test = create_physics_edge_index(n_nodes)

print(f"\n↗️ SINGLE GRAPH SPECIFICATIONS:")
print(f"   x.shape:          [1, {n_nodes}] (M_B + {len(feature_cols)} features)")
print(f"   edge_index.shape: {edge_index_test.shape}")
print(f"   edge_attr.shape:  [{edge_index_test.shape[1]}, 6]")
print(f"   edge_index.max(): {edge_index_test.max().item()} ✓ (valid!)")

print(f"\n✅ CELL 3A COMPLETE!")
print(f"✅ Ready for {n_nodes}-node CfA3+SH0ES graphs!")
print(f"✅ Uses ALL your UBVRI + errors + distances + decline rates!")


🔬 LOADING ALL-PARAMETER FUNCTIONS...
📊 Features: ['V_abs_mag', 'U_abs_mag', 'B_dm15', 'R_abs_mag', 'I_abs_mag']... (total 5)
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓

↗️ SINGLE GRAPH SPECIFICATIONS:
   x.shape:          [1, 6] (M_B + 5 features)
   edge_index.shape: torch.Size([2, 30])
   edge_attr.shape:  [30, 6]
   edge_index.max(): 5 ✓ (valid!)

✅ CELL 3A COMPLETE!
✅ Ready for 6-node CfA3+SH0ES graphs!
✅ Uses ALL your UBVRI + errors + distances + decline rates!


In [90]:
# CELL 3B-ULTIMATE-FIX: 25 NODES × 1 FEATURE (PERFECT!)..new
print("🔧 ULTIMATE GRAPH FIX: 25 NODES × 1 FEATURE!")
print(f"📊 train_features: {train_features.shape}")

n_nodes = len(feature_cols) + 1  # M_B + all features
train_graphs = []
test_graphs = []

for split_name, features, targets in [
    ("TRAIN", train_features, train_target),
    ("TEST", test_features, test_target)
]:
    print(f"\n🔨 Building {len(features)} {split_name} graphs...")

    for i in range(len(features)):
        # ✅ CORRECT: n_nodes, each with 1 feature
        # Nodes 0 to n_features-1: features, Node n_features: M_B target
        x = torch.zeros(n_nodes, 1)  # [n_nodes, 1]
        x[:len(feature_cols), 0] = features[i]  # Features → nodes 0 to n_features-1
        x[-1, 0] = targets[i]  # M_B → node n_features

        # ✅ FIXED: edge_index matches n_nodes exactly
        edge_index = create_physics_edge_index(n_nodes)  # [2, n_edges], indices [0 to n_nodes-1]

        # Edge physics (truncate to exact edge count)
        # Note: templates[0] is just a placeholder formula for edge feature computation
        # The actual formula coefficients are learned later.
        edge_attr = compute_physics_edge_features(
            targets, features, templates[0], feature_cols
        )[:edge_index.shape[1]]  # [n_edges, 6]

        # Ensure edge_attr is 2D, even if compute_physics_edge_features returns a 1D tensor
        if edge_attr.dim() == 1:
            # This case ideally shouldn't happen with the current compute_physics_edge_features,
            # but it's a safeguard if only one edge feature is generated.
            # It would repeat the single feature vector for all edges, which is not ideal
            # but prevents dimension errors.
            edge_attr = edge_attr.unsqueeze(0).repeat(edge_index.shape[1], 1)

        graph = Data(
            x=x,              # [n_nodes,1] ✓ n_nodes feature nodes
            edge_index=edge_index,  # [2,n_edges] ✓ indices [0 to n_nodes-1]
            edge_attr=edge_attr,    # [n_edges,6] ✓
            y=targets[i]      # scalar
        )

        if split_name == "TRAIN":
            train_graphs.append(graph)
        else:
            test_graphs.append(graph)

        if i < 1:
            print(f"  {split_name} SN {i}: x={x.shape}, edge_max={edge_index.max()}")

print(f"\n✅ ULTIMATE FIX COMPLETE!")
print(f"   x.shape: [{n_nodes},1] ✓ ({n_nodes} feature nodes)")
print(f"   edge_index: {edge_index.shape}, max={edge_index.max().item()} ✓ (matches {n_nodes} nodes)")
print(f"📊 {len(train_graphs)} TRAIN + {len(test_graphs)} TEST ✓")

🔧 ULTIMATE GRAPH FIX: 25 NODES × 1 FEATURE!
📊 train_features: torch.Size([78, 5])

🔨 Building 78 TRAIN graphs...
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
  TRAIN SN 0: x=torch.Size([6, 1]), edge_max=5
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_index: torch.Size([2, 30]), range=[0-5] ✓
✅ edge_

In [80]:
# CELL 4F: GNN LEARNS 10K FORMULA COEFFICIENTS (25-NODE PHYSICS!)..new
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import TransformerConv, global_add_pool

print("🚀 GNN learns 10K coefficient sets → ranks by SN error!")
print(f"📊 Your data: x=[1, {len(feature_cols)+1}], edge_index=[2,{((len(feature_cols)+1)*(len(feature_cols)))/2*2}], edge_attr=[{((len(feature_cols)+1)*(len(feature_cols)))/2*2},6]")
print(f"🎯 {len(templates)} formulas × {len(feature_cols)} params")

class MultiFormulaGNN(torch.nn.Module):
    def __init__(self, input_dim=1, edge_dim=6, hidden_dim=128, n_templates=10000, n_params=7):
        super().__init__()
        # Physics message passing (YOUR 600 edges × 6D physics!)
        self.conv1 = TransformerConv(input_dim, hidden_dim//2, heads=4, edge_dim=edge_dim)
        self.conv2 = TransformerConv(hidden_dim//2 * 4, hidden_dim, heads=2, edge_dim=edge_dim)
        self.conv3 = TransformerConv(hidden_dim * 2, hidden_dim//2, edge_dim=edge_dim)
        self.pool = global_add_pool

        self.n_templates = n_templates
        self.n_params = n_params

        # ✅ KEY: PER-FORMULA COEFFICIENTS [n_templates × n_params]
        # Changed to a single nn.Parameter tensor
        self.formula_coeffs = nn.Parameter(torch.randn(n_templates, n_params) * 0.01)

        # Template embeddings are no longer explicitly used for prediction here, but the head maps SN embedding to param weights
        self.head = torch.nn.Linear(hidden_dim//2, n_params)

    def forward(self, x, edge_index, edge_attr, batch):
        # 1. PHYSICS MESSAGE PASSING (your 600 edges!)
        x1 = F.elu(self.conv1(x, edge_index, edge_attr))
        x2 = F.elu(self.conv2(x1, edge_index, edge_attr))
        x3 = F.elu(self.conv3(x2, edge_index, edge_attr))

        # 2. SUPERNOVA EMBEDDING [N_graphs, hidden_dim//2]
        sn_emb = self.pool(x3, batch) # [batch_size, hidden_dim//2]

        # 3. PER-SN PARAMETER WEIGHTS (backprop learns these!)
        sn_weights = self.head(sn_emb)  # [N_graphs, n_params]

        # 4. APPLY ALL FORMULAS: M_B_pred = params @ coeffs.T
        # This uses a single matrix multiplication, which is much faster
        predictions = sn_weights @ self.formula_coeffs.T  # [N_graphs, n_templates]

        # Return predictions and the formula coefficients tensor
        return predictions, self.formula_coeffs  # [N_graphs, n_templates], [n_templates, n_params]

# YOUR MODEL (perfect dimensions!)
model = MultiFormulaGNN(
    input_dim=1,           # ← YOUR x.shape[1]=1 ✓
    edge_dim=6,             # ← YOUR edge_attr.shape[1]=6 ✓
    n_templates=len(templates),  # Number of formulas
    n_params=len(feature_cols)   # Number of feature columns
)

# NOTE: Optimizer will now act on all parameters
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(f"✅ MultiFormulaGNN READY!")
print(f"   Input:     x=[{len(feature_cols)+1},1], edge_index=[2,{((len(feature_cols)+1)*(len(feature_cols)))/2*2}], edge_attr=[{((len(feature_cols)+1)*(len(feature_cols)))/2*2},6]")
print(f"   Physics:   3×TransformerConv over {((len(feature_cols)+1)*(len(feature_cols)))/2*2} physics edges")
print(f"   Output:    [N_SNe, {len(templates)}] predictions + [{len(templates)}×{len(feature_cols)}] coefficients")
print(f"🎯 Learns: Optimal (a_i,b_i,...) for ALL {len(templates)} formulas per SN!")


🚀 GNN learns 10K coefficient sets → ranks by SN error!
📊 Your data: x=[1, 6], edge_index=[2,30.0], edge_attr=[30.0,6]
🎯 783 formulas × 5 params
✅ MultiFormulaGNN READY!
   Input:     x=[6,1], edge_index=[2,30.0], edge_attr=[30.0,6]
   Physics:   3×TransformerConv over 30.0 physics edges
   Output:    [N_SNe, 783] predictions + [783×5] coefficients
🎯 Learns: Optimal (a_i,b_i,...) for ALL 783 formulas per SN!


In [61]:
# CELL 4G: GNN TRAINS 10K FORMULAS - SINGLE GRAPH TRAINING!
import torch
import torch.nn.functional as F

print("🔬 Training 10K formulas → NO BATCHING → ZERO index errors!")
print(f"📊 Graphs: x=[1,{len(feature_cols)+1}], edge_index=[2,{((len(feature_cols)+1)*(len(feature_cols)))/2*2}], edge_attr=[{((len(feature_cols)+1)*(len(feature_cols)))/2*2},6]")
print(f"🎯 {len(templates)} formulas × {len(feature_cols)} params")

# --- MODIFIED: Use a single optimizer for all model parameters ----
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) # MODIFIED: Reduced learning rate to 0.0001
# --- END MODIFICATION ----

model.train()
best_total_rmse = float('inf')
best_formula_epoch = -1
best_overall_formula = ""
best_overall_coeffs = None

for epoch in range(5): # MODIFIED: Changed from 200 to 5 epochs for quick verification
    epoch_losses = []

    # Iterate over each graph (supernova)
    for graph_idx, graph in enumerate(train_graphs):

        optimizer.zero_grad()

        predictions, all_coeffs = model(
            graph.x,
            graph.edge_index,
            graph.edge_attr,
            None # NO batching for individual graphs
        )

        true_mb = graph.x[len(feature_cols), 0].item()  # Target is the last node
        # Create a target tensor of shape [1, N_templates] to match predictions shape [1, N_templates]
        targets = torch.full((1, len(templates)), true_mb, device=graph.x.device)

        # --- MODIFIED: Calculate a single loss for all formulas ---
        # This loss will backpropagate to both GNN layers and all formula coefficients
        formula_errors = F.mse_loss(predictions, targets, reduction='none') # [1, N_templates]
        loss = formula_errors.mean() # Mean of all formula errors

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        # --- END MODIFICATION ---

        epoch_losses.append(loss.item()) # Append the overall loss for monitoring

    # VALIDATION (every 1 epoch, to see progress quickly)
    if epoch % 1 == 0: # MODIFIED: Check every epoch for quick feedback
        model.eval()
        test_rmses = []

        with torch.no_grad():
            for graph in test_graphs:
                pred, _ = model(graph.x, graph.edge_index, graph.edge_attr, None)
                true_mb_scalar = graph.x[len(feature_cols), 0].item()
                true_mb_tensor = torch.full_like(pred, fill_value=true_mb_scalar)
                rmse_per_formula = torch.sqrt(F.mse_loss(pred, true_mb_tensor, reduction='none'))
                test_rmses.append(rmse_per_formula)

        all_test_rmses = torch.stack(test_rmses).mean(0)  # [N_templates]
        avg_train_loss = torch.tensor(epoch_losses).mean().item()
        avg_gnn_rmse = torch.sqrt(torch.tensor(epoch_losses).mean()).item() # Calculate RMSE from the averaged MSE loss

        best_formula_rmse_current_epoch = all_test_rmses.min().item()
        best_formula_idx_current_epoch = all_test_rmses.argmin().item()

        print(f"Epoch {epoch:3d}: Train RMSE={avg_gnn_rmse:.3f} | "
              f"Best formula (test) #{best_formula_idx_current_epoch}: {best_formula_rmse_current_epoch:.3f} mag")

        if best_formula_rmse_current_epoch < best_total_rmse:
            best_total_rmse = best_formula_rmse_current_epoch
            best_formula_epoch = epoch
            # Use the global model.formula_coeffs, which is now a single tensor
            best_overall_coeffs = all_coeffs[best_formula_idx_current_epoch].detach().cpu()
            best_overall_formula = templates[best_formula_idx_current_epoch]

        model.train()

        # Show best formula more frequently with reduced epochs
        if epoch % 1 == 0: # MODIFIED: Show best formula every epoch
            print(f"   🏆 Best overall formula so far (from epoch {best_formula_epoch}): '{best_overall_formula}'")
            if best_overall_coeffs is not None:
                print(f"   ▀ Coeffs: {best_overall_coeffs[:len(feature_cols)].numpy()}")

    # Early stopping (adjusted for shorter run)
    # Removed early stopping condition due to very low epoch count for verification
    # if epoch > 2 and avg_gnn_rmse < 0.1:
    #     print(f"✨ EARLY STOP: RMSE={avg_gnn_rmse:.3f}")
    #     break

print(f"\n™ FINAL RESULTS!")
print(f"✅ BEST RMSE on test set: {best_total_rmse:.3f} mag (achieved at epoch {best_formula_epoch})")
print(f"™ BEST FORMULA: {best_overall_formula}")
print(f"⚖′  BEST COEFFS: {best_overall_coeffs.numpy()}")
print(f"▀ {len(feature_cols)} params optimized for {len(test_graphs)} SNe!")

🔬 Training 10K formulas → NO BATCHING → ZERO index errors!
📊 Graphs: x=[1,6], edge_index=[2,30.0], edge_attr=[30.0,6]
🎯 100 formulas × 5 params
Epoch   0: Train RMSE=15.093 | Best formula (test) #5: 0.187 mag
   🏆 Best overall formula so far (from epoch 0): 'a*log(B_dm15/V_abs_mag)+b*U_abs_mag'
   ▀ Coeffs: [-0.00194051  0.01083742 -0.01925258  0.00668959 -0.01128357]
Epoch   1: Train RMSE=3.488 | Best formula (test) #93: 0.183 mag
   🏆 Best overall formula so far (from epoch 1): 'a*V_abs_mag**0.5*I_abs_mag+c*U_abs_mag'
   ▀ Coeffs: [ 0.00038385  0.00200078 -0.01735904  0.00452723 -0.02302185]
Epoch   2: Train RMSE=0.440 | Best formula (test) #99: 0.183 mag
   🏆 Best overall formula so far (from epoch 2): 'a*(R_abs_mag-1.0)'
   ▀ Coeffs: [ 0.02593621  0.01001261 -0.01145363 -0.01461704  0.00386235]
Epoch   3: Train RMSE=0.389 | Best formula (test) #22: 0.199 mag
   🏆 Best overall formula so far (from epoch 2): 'a*(R_abs_mag-1.0)'
   ▀ Coeffs: [ 0.0259342   0.01001474 -0.01145172 -0.014

I want this code to modify a bit….

assume i have frmula 1…this will go to GNN firstly for SN1 data…it will learn and then get the weight and coefficient..and now it will calculate and find the error…to make it minimum a NN does..it does the backpropagation to minimize te error ..and so adjust the weight which eventually also adjust the corfficent of the formula…finally we will get the minimum error for SN1…for that same formula..repeat for all SN…and fid the best suiting global coefficient for that formula….

now do the same process for all formula available…

at last the one with the best least error will be our best emperical fomula…

In [92]:
# CELL 4F+4G: PERFECT DIMENSION MATCH - COMPLETE!
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import TransformerConv, global_add_pool
import re
import numpy as np

print("🚀 SEQUENTIAL FORMULA TRAINING - DIMENSION SAFE!")
print(f"📊 Nodes: {len(feature_cols)+1}, Formulas: {len(templates)}")

def count_real_coefficients(formula):
    """Count EXACT a*, b*, c* coefficients"""
    coeffs = re.findall(r'[a-d]\*', formula)
    return max(1, len(coeffs))

def get_formula_features_safe(formula, feature_cols, node_features, n_coeffs):
    """🚨 SAFETY: Always return EXACTLY n_coeffs features"""
    # Parse parameters from formula
    params = re.findall(r'[A-Za-z_][A-Za-z0-9_]*', formula)
    params = [p for p in params if p not in ['a','b','c','d','log','exp','min','abs']]

    formula_features = torch.zeros(n_coeffs, device=node_features.device)

    # Map FIRST n_coeffs params to features (pad with zeros if needed)
    for i in range(n_coeffs):
        if i < len(params) and params[i] in feature_cols:
            param_idx = feature_cols.index(params[i])
            formula_features[i] = node_features[param_idx]
        # else: use 0.0 (padding)

    return formula_features  # ALWAYS [n_coeffs] ✓

class SafeFormulaGNN(nn.Module):
    def __init__(self, input_dim=1, edge_dim=6, hidden_dim=128, n_coeffs=2):
        super().__init__()
        self.conv1 = TransformerConv(input_dim, hidden_dim//2, heads=4, edge_dim=edge_dim)
        self.conv2 = TransformerConv(hidden_dim//2*4, hidden_dim, heads=2, edge_dim=edge_dim)
        self.conv3 = TransformerConv(hidden_dim*2, hidden_dim//2, edge_dim=edge_dim)
        self.pool = global_add_pool
        self.head = nn.Linear(hidden_dim//2, n_coeffs)

    def forward(self, x, edge_index, edge_attr):
        x1 = F.elu(self.conv1(x, edge_index, edge_attr))
        x2 = F.elu(self.conv2(x1, edge_index, edge_attr))
        x3 = F.elu(self.conv3(x2, edge_index, edge_attr))
        sn_emb = self.pool(x3, torch.zeros(x.size(0), dtype=torch.long, device=x.device))
        coeffs = self.head(sn_emb).squeeze(0)  # [n_coeffs]
        return coeffs

# 🔥 MAIN TRAINING LOOP
print("\n🔬 TRAINING: Guaranteed dimension match!")
best_global_rmse = float('inf')
best_formula_idx, best_coeffs, best_formula = -1, None, ""
all_formula_rmses = []

for formula_idx in range(min(100, len(templates))):  # First 20 for speed
    formula = templates[formula_idx]
    n_coeffs = count_real_coefficients(formula)

    print(f"\n--- FORMULA #{formula_idx+1}/100 ---")
    print(f"📝 '{formula}' → {n_coeffs} coeffs")

    # Fresh GNN for this formula
    model = SafeFormulaGNN(input_dim=1, edge_dim=6, n_coeffs=n_coeffs)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # TRAIN: All 78 SNe
    model.train()
    for epoch in range(25):  # Faster training
        for graph in train_graphs:
            optimizer.zero_grad()

            coeffs = model(graph.x, graph.edge_index, graph.edge_attr)  # [n_coeffs]
            node_features = graph.x[:-1, 0]  # [6]

            # 🚨 SAFETY: EXACT dimension match!
            formula_features = get_formula_features_safe(formula, feature_cols, node_features, n_coeffs)
            assert coeffs.shape == formula_features.shape, f"Shape mismatch: {coeffs.shape} vs {formula_features.shape}"

            pred_mb = (coeffs * formula_features).sum()
            true_mb = graph.x[-1, 0]

            loss = F.mse_loss(pred_mb, true_mb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

    # TEST: All 34 test SNe
    model.eval()
    test_preds, test_trues = [], []
    with torch.no_grad():
        for graph in test_graphs:
            coeffs = model(graph.x, graph.edge_index, graph.edge_attr)
            node_features = graph.x[:-1, 0]
            formula_features = get_formula_features_safe(formula, feature_cols, node_features, n_coeffs)

            pred_mb = (coeffs * formula_features).sum()
            test_preds.append(pred_mb.item())
            test_trues.append(graph.x[-1, 0].item())

    # Calculate RMSE
    test_rmse = np.sqrt(np.mean((np.array(test_preds) - np.array(test_trues))**2))
    all_formula_rmses.append(test_rmse)

    print(f"✅ RMSE: {test_rmse:.4f} mag")
    print(f"🏋️  Coeffs: {coeffs.cpu().numpy().round(4)}")
    print(f"🔍 Features used: {formula_features.cpu().numpy().round(4)}")

    # Track best
    if test_rmse < best_global_rmse and not np.isnan(test_rmse):
        best_global_rmse = test_rmse
        best_formula_idx = formula_idx
        best_coeffs = coeffs.cpu().numpy()
        best_formula = formula
        print(f"🥇 NEW BEST!")

print(f"\n{'='*70}")
print(f"🎉 CHAMPION FORMULA!")
print(f"🏆 #{best_formula_idx+1}: '{best_formula}'")
print(f"⚡ RMSE: {best_global_rmse:.4f} mag")
print(f"📊 Coeffs: {best_coeffs.round(4)}")

# Top 5
if len(all_formula_rmses) > 1:
    top5 = np.argsort(all_formula_rmses)[:5]
    print(f"\n📈 TOP 5:")
    for i, idx in enumerate(top5):
        print(f"{i+1}. '{templates[idx]}' → {all_formula_rmses[idx]:.4f}")


🚀 SEQUENTIAL FORMULA TRAINING - DIMENSION SAFE!
📊 Nodes: 6, Formulas: 783

🔬 TRAINING: Guaranteed dimension match!

--- FORMULA #1/100 ---
📝 'a*exp(B_dm15/5)' → 1 coeffs
✅ RMSE: 3.8846 mag
🏋️  Coeffs: [-16.7241]
🔍 Features used: [1.3187]
🥇 NEW BEST!

--- FORMULA #2/100 ---
📝 'a*U_abs_mag*B_dm15**0.5' → 1 coeffs
✅ RMSE: 0.4763 mag
🏋️  Coeffs: [1.0021]
🔍 Features used: [-18.5469]
🥇 NEW BEST!

--- FORMULA #3/100 ---
📝 'a*U_abs_mag**2*I_abs_mag' → 1 coeffs
✅ RMSE: 0.4744 mag
🏋️  Coeffs: [0.9774]
🔍 Features used: [-18.5469]
🥇 NEW BEST!

--- FORMULA #4/100 ---
📝 '-a*U_abs_mag+b*(R_abs_mag+V_abs_mag)a*log(U_abs_mag/R_abs_mag)+b*V_abs_mag' → 4 coeffs
✅ RMSE: 2.4268 mag
🏋️  Coeffs: [-0.7372 -0.834   2.3031  0.1291]
🔍 Features used: [-18.5469 -18.7567 -18.522  -18.5469]

--- FORMULA #5/100 ---
📝 'a*I_abs_mag**0.5*U_abs_mag+c*R_abs_mag' → 2 coeffs
✅ RMSE: 1.1444 mag
🏋️  Coeffs: [ 1.4799 -0.4374]
🔍 Features used: [-18.602  -18.5469]

--- FORMULA #6/100 ---
📝 'a*(I_abs_mag/B_dm15)+b*R_abs_mag' → 2 

In [ ]:
print(
  --- FORMULA #1/10 ---
📝 a*U_abs_mag+b*(B_dm15-V_abs_mag) → 2 coeffs
✅ RMSE: 0.5373 mag
🏋️ Coeffs: [ 0.9758 -0.5456]
🔍 Features used: [-18.5469 1.3187]
🥇 NEW BEST!")